In [1]:
import os
import json
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime, timezone, timedelta
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
# 新增：导入准确率评估指标
from sklearn.metrics import accuracy_score

# ===================== 核心配置（仅需修改这1个路径） =====================
CORE_FOLDER = "/Users/shiyiwen/Desktop/科研/数据提取代码/Kickstarter_2025-12-18T03_20_24_296Z"

# 定义需要检查缺失的核心字段（可根据实际需求增减）
CHECK_MISSING_FIELDS = {
    "deadline": "截止时间",
    "launched_at": "发布时间",
    "goal": "筹款目标",
    "fx_rate": "汇率",
    "country": "国家",
    "category": "分类信息",
    "name": "项目名称",
    "id": "项目ID"
}

# 特征名称映射（改为英文，适配图表）
FEATURE_NAME_MAP = {
    "duration_days": "Funding Duration (days)",
    "cat_enc": "Project Category",
    "country_enc": "Country",
    "goal_usd": "Funding Goal (USD)",
    "fund_sim": "Goal Reasonableness",
    "time_ratio": "Duration Ratio (vs 7 days)"
}

# ===================== 初始化Matplotlib =====================
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False 
# ===================== 核心类：全量CSV训练 + 全量CSV预测 =====================
class KSFullCSVProcessor:
    def __init__(self):
        # 1. 自动创建模型子文件夹，统一存放模型文件
        self.model_dir = os.path.join(CORE_FOLDER, "model_output")
        os.makedirs(self.model_dir, exist_ok=True)
        
        # 2. 自动创建预测结果子文件夹，存放单个文件的预测结果
        self.pred_result_dir = os.path.join(CORE_FOLDER, "单个文件预测结果")
        os.makedirs(self.pred_result_dir, exist_ok=True)
        
        # 模型文件路径改为子文件夹内
        self.model_files = {
            "model": os.path.join(self.model_dir, "ks_model.pkl"),
            "scaler": os.path.join(self.model_dir, "ks_scaler.pkl"),
            "feats": os.path.join(self.model_dir, "ks_features.json"),
            "le_cat": os.path.join(self.model_dir, "ks_le_cat.pkl"),
            "le_country": os.path.join(self.model_dir, "ks_le_country.pkl"),
            "goal_median": os.path.join(self.model_dir, "ks_goal_median.json"),
            "feature_importance": os.path.join(self.model_dir, "feature_importance.json"),
            # 新增：保存模型准确率的文件路径
            "model_accuracy": os.path.join(self.model_dir, "model_accuracy.json")  
        }

    # -------------------------- 获取实时北京时间时间戳 --------------------------
    def _get_beijing_time_timestamp(self):
        """获取实时北京时间的秒级时间戳（UTC+8）"""
        beijing_tz = timezone(timedelta(hours=8))
        now = datetime.now(beijing_tz)
        return int(now.timestamp())

    # -------------------------- 精准识别缺失字段 --------------------------
    def _get_missing_fields_detail(self, row):
        """
        识别单条数据中缺失的具体字段，返回可读的缺失描述
        :param row: DataFrame的单行数据
        :return: 缺失字段描述（如"缺失截止时间、缺失国家"或"无缺失"）
        """
        missing_fields = []
        for field_code, field_name in CHECK_MISSING_FIELDS.items():            # 检查字段是否存在缺失（NaN/空字符串/0（仅针对非数值字段））
            if pd.isna(row[field_code]):
                missing_fields.append(field_name)
            elif isinstance(row[field_code], str) and row[field_code].strip() == "":
                missing_fields.append(field_name)
            # 对数值型关键字段（如id、goal），0也视为缺失
            elif field_code in ["id", "goal"] and row[field_code] == 0:
                missing_fields.append(field_name)
        
        if not missing_fields:
            return "无缺失"
        else:
            return "、".join([f"缺失{field}" for field in missing_fields])

    # -------------------------- 原有工具函数 --------------------------
    def _safe_ts_convert(self, ts):
        if pd.isna(ts) or str(ts).lower() in ["nan", "", "None"]:
            return datetime(2020, 1, 1)
        try:
            ts_int = int(float(ts))
            ts_int = ts_int // 1000 if ts_int > 1e12 else ts_int
            return datetime.fromtimestamp(ts_int)
        except:
            return datetime(2020, 1, 1)

    def _safe_parse_category(self, cat_str):
        if pd.isna(cat_str) or str(cat_str).lower() in ["nan", "", "None"]:
            return "OTHER"
        try:
            cat_dict = json.loads(cat_str.replace("'", '"').strip())
            return cat_dict.get("parent_name", cat_dict.get("name", "OTHER")).strip()
        except:
            return "OTHER"    # -------------------------- 提取并保存特征重要性（英文图表） --------------------------
    def _extract_feature_importance(self, model, feature_cols):
        """
        从随机森林模型中提取特征重要性，保存并返回可视化的结果（英文图表）
        :param model: 训练好的随机森林模型
        :param feature_cols: 特征列名列表
        :return: 特征重要性字典（英文名称: 重要性值）
        """
        # 获取特征重要性
        importances = model.feature_importances_
        # 构建特征重要性字典（代码特征名 → 英文业务名称 → 重要性值）
        feat_imp_dict = {}
        for col, imp in zip(feature_cols, importances):
            feat_imp_dict[FEATURE_NAME_MAP.get(col, col)] = round(imp, 4)
        
        # 按重要性降序排序
        feat_imp_sorted = dict(sorted(feat_imp_dict.items(), key=lambda x: x[1], reverse=True))
        
        # 保存到文件
        with open(self.model_files["feature_importance"], "w") as f:
            json.dump(feat_imp_sorted, f, ensure_ascii=False, indent=4)
        
        # 绘制英文图表
        plt.figure(figsize=(10, 6))
        plt.bar(feat_imp_sorted.keys(), feat_imp_sorted.values(), color='#3498db')
        plt.title("Core Factors Affecting Project Success/Failure (Feature Importance)", fontsize=12)
        plt.xlabel("Influencing Factors", fontsize=10)
        plt.ylabel("Importance Value", fontsize=10)
        plt.xticks(rotation=45, ha='right')  
        plt.tight_layout()  
        # 保存图片到model_output文件夹
        plt.savefig(os.path.join(self.model_dir, "feature_importance_analysis.png"), dpi=300)
        plt.close()
        
        return feat_imp_sorted
        # -------------------------- 分析失败影响因素 --------------------------
    def _analyze_failure_factors(self, df_train, feat_imp_sorted):
        """
        结合特征重要性和训练数据，分析“哪些因素导致项目失败”（英文结论）
        :param df_train: 训练数据集
        :param feat_imp_sorted: 排序后的特征重要性字典
        :return: 失败影响因素的英文结论
        """
        failure_factors = []
        # 1. 分析筹款目标的影响
        top_feat = list(feat_imp_sorted.keys())[0]
        if top_feat == "Funding Goal (USD)":
            # 计算失败/成功项目的筹款目标均值
            fail_goal_mean = df_train[df_train["label"] == 0]["goal_usd"].mean()
            succ_goal_mean = df_train[df_train["label"] == 1]["goal_usd"].mean()
            failure_factors.append(f"✅ Core Failure Factor: Excessively high funding goal (Failed projects: ${round(fail_goal_mean, 2)}, Successful projects: ${round(succ_goal_mean, 2)})")
        
        # 2. 分析众筹周期的影响
        if "Funding Duration (days)" in feat_imp_sorted:
            fail_duration_mean = df_train[df_train["label"] == 0]["duration_days"].mean()
            succ_duration_mean = df_train[df_train["label"] == 1]["duration_days"].mean()
            failure_factors.append(f"✅ Important Failure Factor: Unreasonable funding duration (Failed projects: {round(fail_duration_mean, 1)} days, Successful projects: {round(succ_duration_mean, 1)} days)")
        
        # 3. 分析分类/国家的影响
        if "Project Category" in feat_imp_sorted:
            failure_factors.append(f"✅ Important Failure Factor: Project category (Significant differences in failure rates across categories)")
        if "Country" in feat_imp_sorted:
            failure_factors.append(f"✅ Important Failure Factor: Country (Large differences in success rates across regions)")
        
        # 4. 补充字段缺失的影响（非模型特征，但业务上重要）
        failure_factors.append(f"✅ Critical Failure Factor: Missing core fields (e.g., deadline/goal missing directly reduces success rate)")
        
        return failure_factors

    # -------------------------- 步骤1：全量CSV训练 --------------------------    
    def full_csv_train(self):
        print("📌 Starting training (reading all CSV files in core folder)...")
        all_csv_files = [f for f in os.listdir(CORE_FOLDER) if f.endswith(".csv")]
        if not all_csv_files:
            raise ValueError(f"❌ No CSV files found in core folder: {CORE_FOLDER}!")
        
        df_list = []
        for file in all_csv_files:
            file_path = os.path.join(CORE_FOLDER, file)
            try:
                df = pd.read_csv(file_path, encoding="utf-8")
            except:
                df = pd.read_csv(file_path, encoding="gbk")
            df_list.append(df)
            print(f"✅ Loaded training file: {file} (Records: {len(df)})")
        df_train = pd.concat(df_list, ignore_index=True)
        print(f"\n📊 Total training data after merging: {len(df_train)} records\n")

        df_train = df_train[df_train["state"].str.lower().isin(["successful", "failed"])].copy()
        df_train["label"] = df_train["state"].map({"successful": 1, "failed": 0})

        df_train["fx_rate"] = df_train["fx_rate"].fillna(1.0)
        df_train["goal"] = df_train["goal"].fillna(0)
        df_train["country"] = df_train["country"].fillna("OTHER")
        df_train["category"] = df_train["category"].fillna('{"name":"OTHER"}')

        df_train["goal_usd"] = df_train["goal"] * df_train["fx_rate"]
        df_train["duration_days"] = df_train.apply(
            lambda x: (self._safe_ts_convert(x["deadline"]) - self._safe_ts_convert(x["launched_at"])).days,
            axis=1
        )
        df_train["main_category"] = df_train["category"].apply(self._safe_parse_category)

        le_cat = LabelEncoder()
        df_train["cat_enc"] = le_cat.fit_transform(df_train["main_category"])
        le_country = LabelEncoder()
        df_train["country_enc"] = le_country.fit_transform(df_train["country"])

        # 修复：训练阶段就确保中位数≥0.01，从根源避免除以0
        goal_median = df_train.groupby("main_category")["goal_usd"].median().to_dict()
        goal_median = {k: max(v, 0.01) for k, v in goal_median.items()}
        
        df_train["goal_reasonable"] = df_train.apply(
            lambda x: x["goal_usd"] / goal_median.get(x["main_category"], 1), axis=1
        ).clip(0, 1000)
        df_train["fund_sim"] = (1 / df_train["goal_reasonable"]) * df_train["cat_enc"]
        df_train["time_ratio"] = df_train["duration_days"].apply(lambda x: max(0.01, (x-7)/max(x, 1)))

        feature_cols = ["duration_days", "cat_enc", "country_enc", "goal_usd", "fund_sim", "time_ratio"]
        with open(self.model_files["feats"], "w") as f:
            json.dump(feature_cols, f)
        with open(self.model_files["goal_median"], "w") as f:
            json.dump(goal_median, f)
        joblib.dump(le_cat, self.model_files["le_cat"])
        joblib.dump(le_country, self.model_files["le_country"])

        # 训练前清理异常值
        X = df_train[feature_cols].fillna(0)
        X = X.replace([np.inf, -np.inf], 0)
        X = X.clip(-1e18, 1e18)
        
        y = df_train["label"]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)  # 提前保存标准化后的特征矩阵
        model = RandomForestClassifier(random_state=42, n_estimators=150)
        model.fit(X_scaled, y)

        joblib.dump(model, self.model_files["model"])        
        joblib.dump(scaler, self.model_files["scaler"])
        
        # ===================== 新增：计算并保存模型整体准确率 =====================
        # 用训练集预测，计算准确率
        y_pred = model.predict(X_scaled)
        model_accuracy = accuracy_score(y, y_pred)
        # 保存准确率到文件
        accuracy_dict = {
            "overall_accuracy": round(model_accuracy, 4),
            "accuracy_percentage": round(model_accuracy * 100, 2),
            "calculation_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        with open(self.model_files["model_accuracy"], "w") as f:
            json.dump(accuracy_dict, f, ensure_ascii=False, indent=4)
        
        # ===================== 提取特征重要性 + 分析失败因素 =====================
        print("\n📊 Extracting feature importance (failure factors)...")
        feat_imp_sorted = self._extract_feature_importance(model, feature_cols)
        failure_factors = self._analyze_failure_factors(df_train, feat_imp_sorted)
        
        # 打印特征重要性和失败因素结论
        print("\n🎯 Feature Importance Ranking (Impact on Success/Failure):")
        for idx, (feat, imp) in enumerate(feat_imp_sorted.items(), 1):
            print(f"   {idx}. {feat}: {imp}")
        
        # 新增：打印模型整体准确率
        print(f"\n📈 Random Forest Model Overall Accuracy:")
        print(f"   - Training Set Accuracy: {accuracy_dict['overall_accuracy']} ({accuracy_dict['accuracy_percentage']}%)")
        
        print("\n📖 Core Failure Factors Analysis:")
        for factor in failure_factors:
            print(f"   {factor}")
        
        print("\n✅ Training completed! Model + feature importance saved to model_output folder\n")

    # -------------------------- 步骤2：单文件预测核心逻辑 --------------------------
    def _predict_single_csv(self, csv_path):
        model = joblib.load(self.model_files["model"])
        scaler = joblib.load(self.model_files["scaler"])
        le_cat = joblib.load(self.model_files["le_cat"])
        le_country = joblib.load(self.model_files["le_country"])
        with open(self.model_files["feats"], "r") as f:
            feature_cols = json.load(f)
        with open(self.model_files["goal_median"], "r") as f:
            goal_median = json.load(f)

        try:
            df_pred = pd.read_csv(csv_path, encoding="utf-8")
        except:
            df_pred = pd.read_csv(csv_path, encoding="gbk")
        file_name = os.path.basename(csv_path)
        print(f"🔍 Processing prediction file: {file_name} (Total projects: {len(df_pred)})")        
        # 拆分id字段的填充逻辑
        fill_defaults = {
            "fx_rate": 1.0,
            "goal": 0,
            "country": "OTHER",
            "category": '{"name":"OTHER"}',
            "launched_at": 1577836800,
            "deadline": self._get_beijing_time_timestamp(),
            "name": "未知项目"
        }
        # 1. 处理其他字段（先填充默认值，再识别缺失）
        for col, val in fill_defaults.items():
            if col not in df_pred.columns:
                df_pred[col] = val
            # 先标记原始缺失值，再填充（避免填充后无法识别）
            df_pred[f"{col}_original"] = df_pred[col]
            df_pred[col] = df_pred[col].fillna(val)
        
        # 2. 单独处理id字段
        if "id" not in df_pred.columns:
            df_pred["id_original"] = np.nan
            df_pred["id"] = list(range(len(df_pred)))
        else:
            df_pred["id_original"] = df_pred["id"]
            df_pred["id"] = df_pred["id"].fillna(0)

        # 计算核心特征
        df_pred["goal_usd"] = df_pred["goal"] * df_pred["fx_rate"]
        df_pred["duration_days"] = df_pred.apply(
            lambda x: (self._safe_ts_convert(x["deadline"]) - self._safe_ts_convert(x["launched_at"])).days,
            axis=1
        )
        df_pred["main_category"] = df_pred["category"].apply(self._safe_parse_category)

        df_pred["cat_enc"] = df_pred["main_category"].apply(            
            lambda x: le_cat.transform([x])[0] if x in le_cat.classes_ else le_cat.transform(["OTHER"])[0]
        )
        df_pred["country_enc"] = df_pred["country"].apply(
            lambda x: le_country.transform([x])[0] if x in le_country.classes_ else le_country.transform(["OTHER"])[0]
        )

        # 分母最小设为0.01，避免除以0产生无穷大
        df_pred["goal_reasonable"] = df_pred.apply(
            lambda x: x["goal_usd"] / max(goal_median.get(x["main_category"], 1), 0.01), axis=1
        ).clip(0, 1000)
        df_pred["fund_sim"] = (1 / df_pred["goal_reasonable"]) * df_pred["cat_enc"]
        df_pred["time_ratio"] = df_pred["duration_days"].apply(lambda x: max(0.01, (x-7)/max(x, 1)))

        # 清理无穷大/异常值，避免模型报错
        X_pred = df_pred[feature_cols].fillna(0)
        X_pred = X_pred.replace([np.inf, -np.inf], 0)
        X_pred = X_pred.clip(-1e18, 1e18)
        
        X_scaled = scaler.transform(X_pred)
        pred_prob = model.predict_proba(X_scaled)[:, 1]
        pred_state = ["successful" if p > 0.5 else "failed" for p in pred_prob]

       
        # 恢复原始字段用于缺失检测
        for field in CHECK_MISSING_FIELDS.keys():
            if f"{field}_original" in df_pred.columns:
                df_pred[field] = df_pred[f"{field}_original"]
                df_pred.drop(columns=[f"{field}_original"], inplace=True)
        
        # 生成缺失字段详情
        df_pred["数据缺失情况"] = df_pred.apply(self._get_missing_fields_detail, axis=1)

        result_df = pd.DataFrame({
            "Project ID": df_pred["id"],  
            "Project Name": df_pred["name"].apply(lambda x: str(x)[:30]+"..." if len(str(x))>30 else str(x)),            
            "Main Category": df_pred["main_category"],
            "Country": df_pred["country"],
            "Funding Goal (USD)": df_pred["goal_usd"].round(2),
            "Funding Duration (days)": df_pred["duration_days"],
            "Predicted Status": pred_state,
            "Success Probability": np.round(pred_prob, 4),
            "Missing Fields": df_pred["数据缺失情况"], 
            "Source File": file_name
        })

        # 单个预测结果保存到「单个文件预测结果」子文件夹
        single_result_path = os.path.join(self.pred_result_dir, f"prediction_result_{file_name}")
        result_df.to_csv(single_result_path, index=False, encoding="utf-8-sig")
        print(f"✅ Prediction completed for {file_name}! Saved to: {single_result_path}\n")
        return result_df

    # -------------------------- 步骤3：全量CSV预测 + 强制合并汇总 --------------------------
    def full_csv_predict(self):
        print("📌 Starting batch prediction (reading all CSV files in core folder)...\n")
        all_csv_files = [f for f in os.listdir(CORE_FOLDER) if f.endswith(".csv")]
        if not all_csv_files:
            raise ValueError(f"❌ No CSV files found in core folder: {CORE_FOLDER}!")
        
        # 过滤掉已生成的预测结果文件，只处理原始数据文件
        pred_files = [f for f in all_csv_files if not f.startswith("prediction_result") and not f.startswith("full_prediction_summary")]
        if not pred_files:
            print("⚠️ No original CSV files found for prediction (filtered result files)")
            return
        
        all_results = []
        processed_files = []
        for file in pred_files:
            file_path = os.path.join(CORE_FOLDER, file)
            try:
                result_df = self._predict_single_csv(file_path)                
                all_results.append(result_df)
                processed_files.append(file)
            except Exception as e:
                print(f"❌ Failed to process {file}: {str(e)}, skipped")
                continue
        
        # 强制合并所有成功处理的结果
        if all_results:
            # 合并所有结果
            merge_df = pd.concat(all_results, ignore_index=True)
            # 清理合并后的异常值（避免空值/特殊字符）
            merge_df = merge_df.fillna("Unknown")
            merge_df = merge_df.replace([np.inf, -np.inf], 0)
            
            # 生成带时间戳的汇总文件（避免覆盖），汇总文件仍保存在核心文件夹
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            merge_result_path = os.path.join(CORE_FOLDER, f"full_prediction_summary_{timestamp}.csv")
            
            # 保存汇总文件（utf-8-sig兼容Excel）
            merge_df.to_csv(merge_result_path, index=False, encoding="utf-8-sig")
            
            # 统计缺失情况（新增）
            missing_stats = merge_df["Missing Fields"].value_counts()
            
            # ===================== 读取特征重要性和模型准确率，加入汇总日志 =====================
            print("\n🎯 Core Factors Affecting Project Failure (Extracted from Model):")
            if os.path.exists(self.model_files["feature_importance"]):
                with open(self.model_files["feature_importance"], "r") as f:
                    feat_imp_sorted = json.load(f)
                for idx, (feat, imp) in enumerate(feat_imp_sorted.items(), 1):
                    print(f"   {idx}. {feat} (Importance: {imp})")
            
            # 新增：打印模型准确率
            if os.path.exists(self.model_files["model_accuracy"]):
                with open(self.model_files["model_accuracy"], "r") as f:
                    accuracy_dict = json.load(f)
                print(f"\n📈 Random Forest Model Performance:")
                print(f"   - Overall Accuracy: {accuracy_dict['overall_accuracy']} ({accuracy_dict['accuracy_percentage']}%)")
            
            print("="*60)
            print(f"✅ Batch prediction completed!")
            print(f"📁 Single prediction results path: {self.pred_result_dir}")            
            print(f"📁 Full summary file path: {merge_result_path}")
            print(f"📊 Summary Statistics:")
            print(f"   - Processed files: {len(processed_files)}")
            print(f"   - Total projects: {len(merge_df)}")
            print(f"   - Predicted successful: {len(merge_df[merge_df['Predicted Status']=='successful'])}")
            print(f"   - Predicted failed: {len(merge_df[merge_df['Predicted Status']=='failed'])}")
            print(f"\n📋 Missing Fields Statistics:")
            for missing_type, count in missing_stats.head(10).items():
                print(f"   - {missing_type}: {count} records")
            print("="*60)
        else:
            print("❌ No files processed successfully, cannot generate summary")

# ===================== 主执行逻辑 =====================
if __name__ == "__main__":
    processor = KSFullCSVProcessor()
    
    try:
        # 先训练（如果模型不存在）
        if not os.path.exists(processor.model_files["model"]):
            processor.full_csv_train()
        # 执行预测+汇总
        processor.full_csv_predict()
    except Exception as e:
        print(f"\n❌ Program execution error: {str(e)}")

📌 Starting training (reading all CSV files in core folder)...
✅ Loaded training file: Kickstarter068.csv (Records: 3190)
✅ Loaded training file: Kickstarter040.csv (Records: 3214)
✅ Loaded training file: Kickstarter054.csv (Records: 3178)
✅ Loaded training file: Kickstarter083.csv (Records: 1302)
✅ Loaded training file: Kickstarter082.csv (Records: 3207)
✅ Loaded training file: Kickstarter055.csv (Records: 3180)
✅ Loaded training file: Kickstarter041.csv (Records: 3196)
✅ Loaded training file: Kickstarter069.csv (Records: 3203)
✅ Loaded training file: Kickstarter057.csv (Records: 3209)
✅ Loaded training file: Kickstarter043.csv (Records: 3195)
✅ Loaded training file: Kickstarter080.csv (Records: 3180)
✅ Loaded training file: Kickstarter081.csv (Records: 3193)
✅ Loaded training file: Kickstarter042.csv (Records: 3203)
✅ Loaded training file: Kickstarter056.csv (Records: 3195)
✅ Loaded training file: Kickstarter052.csv (Records: 3205)
✅ Loaded training file: Kickstarter046.csv (Records: 